In [ ]:
#for importing, formatting and data manipulation
import pandas as pd
import numpy as np
import datetime

#for plotting
import matplotlib, random
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
%matplotlib inline
from upsetplot import plot
import plotly.graph_objects as go
import scipy as sp
import statsmodels.api as sm

## Import and format metadata from lab, and BBMP

### Import lab metadata

In [ ]:
#upload the absorbance per sample data
a260230 = pd.read_csv("/Users/Diana/Documents/escuela/phd/ch2/bb_data/2022/a260230.csv")
a260230 = a260230.dropna(how='all') #drop null rows and columns
a260230.dropna(how='all', axis=1, inplace=True)
a260230 = a260230.replace({pd.NA: np.nan})

In [ ]:
#upload metadata of non size fractionated samples
noSF = pd.read_csv("/Users/Diana/Documents/escuela/phd/ch2/bb_data/2022/noSF.csv")
noSF = noSF.dropna(how='all')
noSF.dropna(how='all', axis=1, inplace=True)
noSF = noSF.replace({pd.NA: np.nan})
noSF = noSF.dropna(how='all') #drop null rows and columns
#uncomment the line below to remove metadata columns
#noSF = noSF[["sampleid", "[DNA]ng/ul", "A260/280", "date"]]

In [ ]:
#upload metadata of size fractionated samples
SF = pd.read_csv("/Users/Diana/Documents/escuela/phd/ch2/bb_data/2022/SF.csv")
SF = SF.dropna(how='all')
SF.dropna(how='all', axis=1, inplace=True)
SF = SF.replace({pd.NA: np.nan})
SF = SF.rename(columns={'Depth Code 1-A, 5-B, 10-C, 60-D': 'depth_code',
                            'Size Code 3um - L 0.2um - S': 'size_code'}) 
#uncomment the line below to remove metadata columns
#SF = SF[["sampleid", "[DNA]ng/ul", "A260/280", "date"]]

#### Renumber dates

In [ ]:
#create a dictionary for months
month_dic = {
    "Jan": 1,
    "Feb": 2,
    "Mar": 3,
    "Apr": 4,
    "May": 5,
    "Jun": 6,
    "Jul": 7,
    "Aug": 8,
    "Sep": 9,
    "Oct": 10,
    "Nov": 11,
    "Dec": 12
}
month_season = {
    "Jan": "Winter",
    "Feb": "Winter",
    "Mar": "Spring",
    "Apr": "Spring",
    "May": "Spring",
    "Jun": "Summer",
    "Jul": "Summer",
    "Aug": "Summer",
    "Sep": "Autumn",
    "Oct": "Autumn",
    "Nov": "Autumn",
    "Dec": "Winter"
}
depth_num = {
    "A": 1,
    "B": 5,
    "C": 10,
    "D": 60,
    "E": 30
}

In [ ]:
def fill_dates(md):
    if 'weekn' not in md:
        md["weekn"] = md["sampleid"].str.extract(r'\.([1-9][0-9]?)[A-E]')
    md['weekn'] = pd.to_numeric(md['weekn'])
    md['date'] = md.groupby(['sampleid','weekn'], sort=False, group_keys=False)['date'].apply(lambda x: x.ffill().bfill())

    #add month to a new column
    md['month_name'] = md['date'].str.split('-').str[1]
    md['year'] = 2022
    md=md[md.year==2022]

    #add month number
    md['month']= md['month_name'].map(month_dic)

    #add day number
    md['day'] = md['date'].str.split('-').str[0]
    md[["year", "month", "day"]] = md[["year", "month", "day"]].apply(pd.to_numeric)

    #remove symbol for better handling of data
    #md.rename(columns={"Week#": "Weekn"}, inplace=True)
    #md.rename(columns={"Depth": "depth"}, inplace=True) #to match dfo

    #change to int to remove decimals from date columns
    md.year = md.year.apply(int)
    md.day = md.day.apply(int)
    md.month = md.month.apply(int)
    #md.depth = md.depth.apply(int)
    #md.weekn = md.weekn.apply(int)

    #change to str to aggregate them into time_string to match dfos formatting of the date
    md.year = md.year.apply(str)
    md.month = md.month.apply(str)
    md.day = md.day.apply(str)

    md["depth_code"] = md["sampleid"].str.extract(r'[1-9][0-9]?([A-E])')
    md['depth']= md['depth_code'].map(depth_num)
    md['depth'] = pd.to_numeric(md['depth'])

    #add leading zero to match date format in dfo metadata
    md['month'] = md['month'].str.zfill(2)
    md['day'] = md['day'].str.zfill(2)

    md['time_string'] = md[['year', 'month', 'day']].agg('-'.join, axis=1)
    
    md["size_code"] = md["sampleid"].str.extract(r'[1-9][0-9]?[A-E]([L-S])')
    md["size_code"] = md["size_code"].fillna('W')
    
    return md

In [ ]:
SF = fill_dates(SF)

In [ ]:
noSF = noSF[noSF['weekn'] < 17]
noSF = fill_dates(noSF)

In [ ]:
#verify which columns are shared between two dataframes
a = np.intersect1d(SF.columns, noSF.columns)
a

In [ ]:
colsnum = ['A260/280', '[DNA]ng/ul', 'depth', 'elution_volume', 'filtration_volume ', 'weekn', 'year']
noSF[colsnum] = noSF[colsnum].apply(pd.to_numeric, errors='coerce', axis=1)
SF[colsnum] = SF[colsnum].apply(pd.to_numeric, errors='coerce', axis=1)

colstr = ['sampleid']
SF[colstr] = SF[colstr].astype("string")

SF = SF.replace({pd.NA: np.nan})
noSF = noSF.replace({pd.NA: np.nan})

mdsf = noSF.merge(SF, on=['A260/280', 'Extracted_By', 'Notes', '[DNA]ng/ul', 'date', 'day',
                           'depth', 'depth_code', 'elution_volume', 'extraction_date',
                           'filtration_volume ', 'month', 'month_name', 'sampleid',
                           'size_code', 'time_string', 'weekn', 'year'], how='outer')

In [ ]:
a = np.intersect1d(mdsf.columns, a260230.columns)
a

In [ ]:
#fill missing cell values with matching column name from other dataframe

a260230["weekn"] = a260230["sampleid"].str.extract(r'\.([1-9][0-9]?)[A-E]')
a260230['weekn'] = pd.to_numeric(a260230['weekn'])
    
a260230 = a260230.fillna(noSF)
mdsf = mdsf.fillna(a260230)

a260230["depth_code"] = a260230["sampleid"].str.extract(r'[1-9][0-9]?([A-E])')
a260230['depth']= a260230['depth_code'].map(depth_num)
a260230['depth'] = pd.to_numeric(a260230['depth'])


mdsf2 = mdsf.merge(a260230, on=['A260/280', '[DNA]ng/ul', 'extraction_date', 'sampleid', 'weekn', 'depth_code','depth'], how='outer')

mdsf["weekn"] = mdsf["sampleid"].str.extract(r'\.([1-9][0-9]?)[A-E]')
mdsf['weekn'] = pd.to_numeric(mdsf['weekn'])
mdsf['date'] = mdsf.groupby(['weekn'], sort=False)['date'].apply(lambda x: x.ffill().bfill())

mdsf = mdsf[mdsf['weekn'] < 17]

In [ ]:
md = mdsf2.copy()

In [ ]:
md.sort_values(by=['weekn', 'depth'],inplace=True)
md = md.fillna(method='ffill')

In [ ]:
md.to_csv('metadata_merged.csv')

### Import and manage BBMP data

#### Metadata __md__ is formatted. It contains 38 columns.
__md__ is the lab's metadata for sampling, extraction and sequencing. \
__dfo_md__ is BBMP remote sensing data (salinity, pH, temperature, density..) \
__bio_niskin__ is nutrient data \
Format __bio_niskin__ data to merge with __md__. __bio_niskin__ is 32 columns, including year, month, day, and depth. __dfo_md__ also has 32 columns, including year_time, month_time, day_time. To merge these data with __md__, we will change the time stamps columns to the same name, and generate a time_string column.

In [ ]:
dfo_md = pd.read_csv("/Users/Diana/Documents/escuela/phd/ch2/bb_data/bbmp_aggregated_profiles.csv")
bio_niskin = pd.read_csv("/Users/Diana/Documents/escuela/phd/ch2/bb_data/BBMP_Data_2022.csv")#
#dfo_metadata_y14 = pd.read_csv("/Users/Diana/Documents/escuela/phd/bb_data/2019/data_export/trim-analysis/dfo_metadata_y14.tsv", sep='\t')

#change to str to aggregate them into time_string
bio_niskin = bio_niskin[bio_niskin.year==2022]
bio_niskin.year = bio_niskin.year.apply(str)
bio_niskin.month = bio_niskin.month.apply(str)
bio_niskin.day = bio_niskin.day.apply(str)
#add leading zero to match date format in dfo metadata
bio_niskin['month'] = bio_niskin['month'].str.zfill(2)
bio_niskin['day'] = bio_niskin['day'].str.zfill(2)

bio_niskin['time_string'] = bio_niskin[['year', 'month', 'day']].agg('-'.join, axis=1)

#make a new column for time_string without the time
dfo_md=dfo_md[dfo_md.year_time==2022]
dfo_md['time_string_time'] = dfo_md['time_string']
dfo_md['time_string'] = dfo_md['time_string'].str.split(' ').str[0]

#renaming columns to ensure correct merging
dfo_md.rename(columns={"depth":"bbmpdepth","pressure": "depth", "year_time": "year", "month_time": "month", "day_time": "day"}, inplace=True)

#change to int to remove decimals from date columns
cols = ['year', 'depth', 'month', 'day']
md[cols] = md[cols].apply(pd.to_numeric, errors='ignore', axis=1)
dfo_md[cols] = dfo_md[cols].apply(pd.to_numeric, errors='ignore', axis=1)
bio_niskin[cols] = bio_niskin[cols].apply(pd.to_numeric, errors='ignore', axis=1)

#drop empty columns and rows
dfo_md.dropna(how='all', axis=1, inplace=True) #empty cols
dfo_md.dropna(how='all', inplace=True) #empty rows

bio_niskin.dropna(how='all', axis=1, inplace=True) #empty cols
bio_niskin.dropna(how='all', inplace=True) #empty rows

#make a season column
md['season'] = ''

for month, season in month_season.items():
    md.loc[md['month_name'] == month, 'season'] = season

bio_niskin data has exact recorded depths, whereas BB sample data is restricted to categories: make a new column to allow for data merging

In [ ]:
depths = np.array([1,5,10,60])

In [ ]:
bio_niskin2= bio_niskin.copy()

In [ ]:
length = 48 #number of weeks for the tile repeat
bio_niskin2['NewDepth'] = pd.DataFrame({'NewDepth': np.tile(depths, length)}) #tile depth categorical values

In [ ]:
bio_niskin2=bio_niskin2.assign(NewDepth=depths[np.arange(len(bio_niskin2)) % len(depths)])

In [ ]:
#order the two depth columns at the end of the dataframe to visually examine
cols_at_end = ['depth', 'NewDepth']
bio_niskin3 = bio_niskin2[[c for c in bio_niskin2 if c not in cols_at_end] 
        + [c for c in cols_at_end if c in bio_niskin2]]

In [ ]:
#rename columns to ensure correct merging
bio_niskin3.rename(columns={'depth': 'truedepth', 'NewDepth': 'depth'}, inplace=True)

In [ ]:
#make merging columns to same type
bio_niskin3[cols] = bio_niskin3[cols].apply(pd.to_numeric, errors='ignore', axis=1)
md[cols] = md[cols].apply(pd.to_numeric, errors='ignore', axis=1)
dfo_md[cols] = md[cols].apply(pd.to_numeric, errors='ignore', axis=1)

In [ ]:
a = np.intersect1d(md.columns, dfo_md.columns)
a

In [ ]:
#preview column types to allow for merging
#pd.set_option('display.max_rows', 35)
#md.dtypes

In [ ]:
#convert merging columns to same type
colsnum = ['day', 'month']
dfo_md[colsnum] = dfo_md[colsnum].apply(pd.to_numeric, errors='coerce', axis=1)
md[colsnum] = md[colsnum].apply(pd.to_numeric, errors='coerce', axis=1)

In [ ]:
#merging party
merged = pd.merge(md, dfo_md, on=['day', 'depth', 'month', 'pH', 'sigmaTheta', 'theta',
                                  'time_string', 'year'], how="left")
allyears = pd.merge(md, dfo_md, on=['day', 'depth', 'month', 'pH', 'sigmaTheta', 'theta',
                                    'time_string', 'year'], how="outer")

In [ ]:
merged = merged[merged['weekn'] < 17]

In [ ]:
merged.to_csv('metadata_niskin.csv')

In [ ]:
#convert column type to numeric for merging
allyears[cols] = allyears[cols].apply(pd.to_numeric, errors='ignore', axis=1)

#merged = merged.drop(index=237) #delete a row with missing information
merged[cols] = merged[cols].apply(pd.to_numeric, axis=1)
bio_niskin3[cols] = bio_niskin3[cols].apply(pd.to_numeric, axis=1)

#add nutrient data
#uncomment the line below if  need access to metadata outside the 16weeks samples in 2022
#preall_md= pd.merge(allyears, bio_niskin3, on=["day", "month", "year", 'depth'], how="outer")
all_md = pd.merge(merged, bio_niskin3, on=["day", "month", "year", 'depth'], how="left")

#split dfs by depth
shallow_depths = [1, 5, 10]
shallow = all_md[all_md["depth"] < 30]
#shallow = shallow.groupby(['year', 'month', "day"]).mean().reset_index()
deep = all_md[all_md.depth == 60]

#split dfs by season
year_season = all_md.groupby(by = ['year','season']).mean().reset_index()

Winter = year_season.loc[year_season['season'] == 'Winter',:]
Spring = year_season.loc[year_season['season'] == 'Spring',:]
Summer = year_season.loc[year_season['season'] == 'Summer',:]
Autumn = year_season.loc[year_season['season'] == 'Autumn',:]

#save output as csv
all_md.to_csv('allmetadata.csv')

## Plotting party

In [ ]:
d1 = all_md[all_md.depth_code == 'A']

In [ ]:
sns.relplot(data=d1, x="weekn", y="Chlorophyll A", color="0.8", linewidth=.75, kind="line")

In [ ]:
sns.relplot(data=d1, x="weekn", y="Phosphate", color="0.8", linewidth=.75, kind="line")

In [ ]:
#find rows with null values at given column
emptynit = merged[merged['depth'].isna()]
emptynit

## Find missing data

In [ ]:
emptynit = merged[merged['temperature'].isna()]

In [ ]:
emptynit

In [ ]:
#plotly seasonal averages figure
fig2 = go.Figure()
for template in ["plotly_white"]:
    fig2.add_trace(go.Scatter(x=Winter['year'], y=Winter['temperature'],
                    mode='lines',
                    name='Winter',
                    marker_color='#838B8B'))
    fig2.add_trace(go.Scatter(x=Spring['year'], y=Spring['temperature'],
                    mode='lines',
                    name='Spring',
                    marker_color='#FFB5C5'))
    fig2.add_trace(go.Scatter(x=Summer['year'], y=Summer['temperature'],
                    mode='lines',
                    name='Summer',
                    marker_color='#87CEFF'))
    fig2.add_trace(go.Scatter(x=Autumn['year'], y=Autumn['temperature'],
                    mode='lines',
                    name='Autumn',
                    marker_color='#FF8000'))
    fig2.update_layout(
    height=800,
    xaxis_title="Years",
    yaxis_title='Temperature in degree',
    title_text='Average Temperature seasonwise over the years',
    template=template)

fig2.show()

In [ ]:
#seaborn season averages plot
sns.lineplot(year_season['year'],year_season['temperature'], hue =year_season["season"])

In [ ]:
plt.style.use('ggplot')

### Detect and plot anomalies in variables

In [ ]:
def detect_anomalies(metadata, df, dpt, yr=all, month=all):
    
    sfd=df[df.depth==dpt]
    
    md_col = sfd[['event_id', metadata, "year", "month"]].copy()
    md_col = md_col[md_col[metadata].notna()]
    if yr != all:
        #mdcol_yr = md_col[md_col.Year == yr]
        mdcol_yr = md_col[md_col['year'].isin(yr)]
    else: 
        mdcol_yr = md_col
        
    if month != all:
        #mdcol_yr = mdcol_yr[mdcol_yr.Month == month]
        mdcol_yr = mdcol_yr[mdcol_yr['month'].isin(month)]
    
    mdcol_yr = mdcol_yr.drop(columns=['year', "month"])
    mdcol_yr = mdcol_yr.set_index(['event_id'])
    
    #modelling time
    outliers_fraction = float(.01)
    scaler = StandardScaler()
    np_scaled = scaler.fit_transform(mdcol_yr.values.reshape(-1, 1))
    data = pd.DataFrame(np_scaled)
    # train isolation forest
    model =  IsolationForest(contamination=outliers_fraction)
    model.fit(data)
    
    #predict data
    mdcol_yr['anomaly'] = model.predict(data)
    
    # visualization
    fig, ax = plt.subplots(figsize=(10,6))
    a = mdcol_yr.loc[mdcol_yr['anomaly'] == -1, [metadata]] #anomaly
    ax.plot(mdcol_yr.index, mdcol_yr[metadata], color='black', label = 'Normal')
    ax.scatter(a.index,a[metadata], color='red', label = 'Anomaly')
    #plt.axvline(36, ls='--')
    plt.legend()
    plt.show();
    #add axes names

In [ ]:
detect_anomalies('Temperature', bio_niskin3, 1, yr={2022}, month={1,2,3,4})

In [ ]:
detect_anomalies('Temperature', bio_niskin3, 5, yr={2022}, month={1,2,3,4})

In [ ]:
detect_anomalies('Temperature', bio_niskin3, 10, yr={2022}, month={1,2,3,4})

In [ ]:
detect_anomalies('Temperature', bio_niskin3, 60, yr={2022}, month={1,2,3,4})